<a href="https://colab.research.google.com/github/Njuii/Machine-Learning-Projects-for-Certification/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-04-24 17:58:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K   879KB/s    in 0.4s    

2023-04-24 17:58:42 (879 KB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2023-04-24 17:58:42--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[=========

In [ ]:
# Converting tsv to df
train_data = pd.read_csv(train_file_path, sep="\t", names=["class", "text"])
test_data = pd.read_csv(test_file_path, sep="\t", names=["class", "text"])

# Changing class from categorical to numeric
train_data['class'].replace(['ham','spam'],[0,1],inplace = True)
test_data['class'].replace(['ham','spam'],[0,1],inplace = True)

# Creating labels
train_labels = train_data.pop('class')
test_labels = test_data.pop('class')

train_labels.head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    1
17    0
18    0
19    0
Name: class, dtype: int64

In [ ]:
# Model

# Text vectorisation Layer
encoder = keras.layers.TextVectorization(output_mode='int')

encoder.adapt(train_data)

# Padding text to make same length
vectorised_train_data = keras.preprocessing.sequence.pad_sequences(encoder(train_data), 250)
vectorised_test_data = keras.preprocessing.sequence.pad_sequences(encoder(test_data), 250)


vectorised_train_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 7632, 2425, 3973,    5,  404,  824,   79,    7, 4156,
          3,   47,   57,    7,  153,   22,    3,  177,   56,  165,   79,
       2256,  107,   26,    3,  302,   23,   43,   13,   16,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
# Defining the model
model = keras.Sequential([
    keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64),
    # keras.layers.LSTM(64),
    # keras.layers.SimpleRNN(64, return_sequences=True),
    keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    # keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, None, 64)          523584    
                                                                 
 bidirectional_8 (Bidirecti  (None, None, 128)         66048     
 onal)                                                           
                                                                 
 dense_42 (Dense)            (None, None, 64)          8256      
                                                                 
 dense_43 (Dense)            (None, None, 1)           65        
                                                                 
Total params: 597953 (2.28 MB)
Trainable params: 597953 (2.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics='accuracy')

In [ ]:
# Training the model
history = model.fit(
    vectorised_train_data, 
    train_labels, 
    epochs=10, 
    validation_split=0.2,
    shuffle = True
    )

Epoch 1/10
105/105 [==============================] - 53s 458ms/step - loss: 0.4011 - accuracy: 0.8672 - val_loss: 0.4053 - val_accuracy: 0.8612
Epoch 2/10
105/105 [==============================] - 40s 378ms/step - loss: 0.3899 - accuracy: 0.8673 - val_loss: 0.4244 - val_accuracy: 0.8613
Epoch 3/10
105/105 [==============================] - 42s 404ms/step - loss: 0.2473 - accuracy: 0.9246 - val_loss: 0.1214 - val_accuracy: 0.9689
Epoch 4/10
105/105 [==============================] - 40s 379ms/step - loss: 0.1526 - accuracy: 0.9601 - val_loss: 0.0988 - val_accuracy: 0.9762
Epoch 5/10
105/105 [==============================] - 39s 376ms/step - loss: 0.0911 - accuracy: 0.9800 - val_loss: 0.0844 - val_accuracy: 0.9796
Epoch 6/10
105/105 [==============================] - 39s 377ms/step - loss: 0.0671 - accuracy: 0.9873 - val_loss: 0.0848 - val_accuracy: 0.9822
Epoch 7/10
105/105 [==============================] - 48s 453ms/step - loss: 0.0580 - accuracy: 0.9894 - val_loss: 0.1114 - val_ac

In [ ]:
# Evaluate the model
results = model.evaluate(vectorised_test_data,test_labels)
print(results)

44/44 [==============================] - 6s 128ms/step - loss: 0.1283 - accuracy: 0.9757
[0.12828968465328217, 0.9757041335105896]


In [ ]:
# Tests - to see how I can implement my encoder
pred_text = "how are you doing today?"
pred_text = pd.DataFrame([pred_text])

vectorised_input = encoder(pred_text)
vectorised_input = keras.preprocessing.sequence.pad_sequences(vectorised_input, 250)
prediction = model.predict(vectorised_input)

float(np.mean(prediction[0]))

1/1 [==============================] - 0s 70ms/step


0.004078678786754608

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = pd.DataFrame([pred_text])

  vectorised_input = encoder(pred_text)
  vectorised_input = keras.preprocessing.sequence.pad_sequences(vectorised_input, 250)
  prediction = float(np.mean(model.predict(vectorised_input)))

  if prediction < 0.5:
    prediction_word = 'ham'
  else:
    prediction_word = 'spam'
  return np.array([prediction, prediction_word])

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 53ms/step
['0.004078678786754608' 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 41ms/step
You passed the challenge. Great job!
